In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import tool
from pydantic import SecretStr
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage


@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    return f"The current weather in {location} is sunny."

@tool
def get_rate(currencies) -> str:
    """Get exchange rate between two currencies. The input should be comma separated value between two currencies, e.g USD,INR"""
    try:
      currency1, currency2 = currencies.split(',')
      return f"The exchange rate from {currency1} to {currency2} is 5.6"
    except ValueError:
      return "Invalid input format. Please provide currencies"


tools = [get_weather, get_rate]
config: RunnableConfig = {"configurable": {"thread_id": 1}}


model = ChatOpenAI(
    base_url="http://localhost:8080/v1",
    api_key=SecretStr("dummy"),  # llama.cpp ignores this
    model="mistral-nemo",
)


agent = initialize_agent(
    tools=tools,
    llm=model,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=False,
)


def call_model(state: MessagesState):
    response = agent.invoke({"input": state["messages"]})
    return {"messages": state["messages"] + [AIMessage(content=response["output"])]}


workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config: RunnableConfig = {"configurable": {"thread_id": "1"}}
query = "Hi, what's the current exchange rate between JPY and NPR? Please call the tool and respond whatever it returns"
input_messages = [HumanMessage(content=query)]

for chunk, metadata in app.stream(
    {"messages": input_messages}, config=config, stream_mode="messages"
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="", flush=True)

I need to get the exchange rate between JPY and NPR.
Action: get_rate
Action Input: JPY,NPRI now know the final answer
Final Answer: The current exchange rate between JPY and NPR is 5.6 NPR per JPY.The current exchange rate between JPY and NPR is 5.6 NPR per JPY.